# IBM Hackathon Challenge

##### Problem Statement: The delivery route optimization is one such problem that most companies ranging from tech giants like
###### amazon and small scale logistics companies deal with.Advanced heuristic algorithmsneed to be looked into to
######  find the near optimal solution of shortest route with least number of capacitated vehicles. 
###### This is a classic capacitated vehicle routing problem.


###### Solution:We wish to explore different heuristics to solve the vehicle routing problem and figure out the optimal solution.
######  The implementation of this project can give huge returns forlarge firms which are looking to 
###### optimise their deliveries.


###### Business Use Case:Primary Target audience is Small to Medium business owners that require delivery route
###### optimization to maximize profits. Eg: Milk/Grocery Distribution Firms. The number of
###### delivery nodes cannot exceed 200 as our algorithm is computationally expensive for
###### larger delivery nodes.


###### Approach: GoogleOR - Route Optimisation


###### Team Members: Aiswarya Bimal,Alby Jose, Manjima Jagadeesh, Fathima, Arsha V Chand , Mohammed Sinan , Rajsree S




### Importing Libraries

In [13]:
#Basic Libraries

import pandas as pd

#Distance Libraries
import osrm
import folium
import polyline

#GoogleOR
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

#API Description 
import requests
import json
import warnings
warnings.filterwarnings("ignore")

### Exploratory Data Analysis

In [14]:
#Enter Depo Coordinates
depo_coordinates = '8.90162924792571, 76.57825308040086'
#Add this to top of dataset

In [15]:
#Enter Vehicle Capacities
vehicle_info = [500,900,700,900]

In [16]:
#DATASET LOADING
dataset = pd.read_excel('delivery_dataset.xlsx',sheet_name=0)
dataset.head()

,Location,Coordinates,Demand
0,Pheonix Cafe,"8.920903315650575, 76.64443519221282",46
1,Kilikollur Hotel,"8.908455210793479, 76.62425508719879",5
2,Kavil Bakery,"8.921101537881269, 76.65602770241966",46
3,Ezhutani Hotel,"8.942463859725402, 76.65850943090344",49
4,CDS Canteen,"8.944660170428694, 76.66834626033314",44


In [17]:
dataset.columns

Index(['Location', 'Coordinates', 'Demand'], dtype='object')

In [18]:
#Add depo to dataset
depo_data = []
depo_data.insert(0, {'Location': "Depo", 'Coordinates': depo_coordinates, 'Demand':0})
data_fnl = pd.concat([pd.DataFrame(depo_data),dataset], ignore_index = True)

In [19]:
data_fnl

,Location,Coordinates,Demand
0,Depo,"8.90162924792571, 76.57825308040086",0
1,Pheonix Cafe,"8.920903315650575, 76.64443519221282",46
2,Kilikollur Hotel,"8.908455210793479, 76.62425508719879",5
3,Kavil Bakery,"8.921101537881269, 76.65602770241966",46
4,Ezhutani Hotel,"8.942463859725402, 76.65850943090344",49
...,...,...,...
94,JSM Bakers,"8.904548574472843, 76.6233292420458",35
95,Golden Loaf Bakers,"8.917232097097783, 76.63261416762029",17
96,Cake&bake,"8.916998915494363, 76.63261416752717",39
97,A N Bakers & Juice Stall,"8.917656063232975, 76.63250687926663",33


In [20]:
# Total Demand of Customers
demand = data_fnl.Demand.to_list()

In [21]:
sum(demand)

2690

In [22]:
data_fnl["Latitude"] = data_fnl.Coordinates.apply(lambda x :  x.split(",")[1].strip())
data_fnl["Longitude"] = data_fnl.Coordinates.apply(lambda x :  x.split(",")[0])

In [23]:
data_fnl

,Location,Coordinates,Demand,Latitude,Longitude
0,Depo,"8.90162924792571, 76.57825308040086",0,76.57825308040086,8.90162924792571
1,Pheonix Cafe,"8.920903315650575, 76.64443519221282",46,76.64443519221282,8.920903315650575
2,Kilikollur Hotel,"8.908455210793479, 76.62425508719879",5,76.62425508719879,8.908455210793479
3,Kavil Bakery,"8.921101537881269, 76.65602770241966",46,76.65602770241966,8.921101537881269
4,Ezhutani Hotel,"8.942463859725402, 76.65850943090344",49,76.65850943090344,8.942463859725402
...,...,...,...,...,...
94,JSM Bakers,"8.904548574472843, 76.6233292420458",35,76.6233292420458,8.904548574472843
95,Golden Loaf Bakers,"8.917232097097783, 76.63261416762029",17,76.63261416762029,8.917232097097783
96,Cake&bake,"8.916998915494363, 76.63261416752717",39,76.63261416752717,8.916998915494363
97,A N Bakers & Juice Stall,"8.917656063232975, 76.63250687926663",33,76.63250687926663,8.917656063232975


In [24]:
data_fnl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Location     99 non-null     object
 1   Coordinates  99 non-null     object
 2   Demand       99 non-null     int64 
 3   Latitude     99 non-null     object
 4   Longitude    99 non-null     object
dtypes: int64(1), object(4)
memory usage: 4.0+ KB


In [25]:
data_fnl.describe()

,Demand
count,99.000000
mean,27.171717
std,14.598043
min,0.000000
25%,14.500000
50%,29.000000
75%,39.500000
max,50.000000


In [27]:
temp = []
for x in range(len(data_fnl)):
    temp.append(f'{data_fnl.Latitude.iloc[x]},{data_fnl.Longitude.iloc[x]}')
locations = ";".join(temp)

In [28]:
#OSRM API to get Distance Matrix

url = "http://router.project-osrm.org/table/v1/driving/"
url2 = f"?annotations=distance"
r = requests.get(url+locations+url2)
response = r.json()
if response["code"]=='Ok':
    distance_mx = response["distances"]

In [30]:
#Define function to scale distance Matrix and covert float to int values
def scale_integer_func(Q):
    return [ [ int(10*x) for x in L ] for L in Q ]

distance_matrix = scale_integer_func(distance_mx) #99x99 Matrix
distance_matrix


[[0,
  110095,
  76935,
  123701,
  126054,
  144315,
  154166,
  123698,
  119611,
  198954,
  154192,
  112550,
  52386,
  42008,
  35424,
  33137,
  30355,
  51177,
  50263,
  18889,
  33960,
  112823,
  83334,
  106110,
  87440,
  80501,
  94568,
  93135,
  92986,
  92158,
  89421,
  90277,
  96785,
  76256,
  115743,
  82654,
  92324,
  56534,
  50618,
  77382,
  37494,
  51575,
  112030,
  99292,
  19405,
  138127,
  61696,
  43736,
  29931,
  10404,
  93231,
  52465,
  55652,
  77436,
  103582,
  55638,
  41254,
  43997,
  50894,
  64104,
  124408,
  60509,
  122710,
  42017,
  3115,
  41746,
  12455,
  63430,
  9709,
  37953,
  39615,
  55818,
  43779,
  41049,
  42650,
  93883,
  30674,
  84273,
  46083,
  77798,
  63151,
  180485,
  64951,
  136469,
  95024,
  82102,
  84999,
  86028,
  88108,
  85792,
  76675,
  91944,
  86544,
  84114,
  72687,
  90538,
  90299,
  90791,
  92759],
 [107167,
  0,
  33160,
  14553,
  34117,
  48004,
  65109,
  45514,
  55803,
  86869,
  12069

#### GoogleOR Algorithm

In [31]:
"""Capacited Vehicles Routing Problem (CVRP)."""

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['demands'] = demand
    data['vehicle_capacities'] = vehicle_info
    data['num_vehicles'] = len(vehicle_info)
    data['depot'] = 0 #index of depo
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
#     print(f'Objective: {solution.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(data_fnl["Location"].iloc[manager.IndexToNode(index)], route_load)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(data_fnl["Location"].iloc[manager.IndexToNode(index)], route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance/10)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance/10))
    print('Total load of all routes: {}'.format(total_load))
    
def save_solution(data, manager, routing, solution):
    total_distance = 0
    total_load = 0
    vehicle_optimized_route = pd.DataFrame(columns=["VehicleID","Optimized_route","Optimized_route_index","Total_load","Total_distance"])
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        routes_fnl=[]
        routes_index = []
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            routes_fnl.append(data_fnl["Location"].iloc[manager.IndexToNode(index)])
            routes_index.append(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
        routes_fnl.append(data_fnl["Location"].iloc[manager.IndexToNode(index)])
        routes_index.append(manager.IndexToNode(index))
        vehicle_optimized_route = vehicle_optimized_route.append({"VehicleID": vehicle_id, \
                                                  "Optimized_route":routes_fnl, \
                                                  "Optimized_route_index": routes_index, \
                                                  "Total_load":route_load, \
                                                  "Total_distance":route_distance}, ignore_index=True)
    return pd.DataFrame(vehicle_optimized_route)
    
    


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    
    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    
    #Add Distance Constraint
    routing.AddDimension(transit_callback_index,
        0,  # null capacity slack
        800000,  # vehicle maximum distance ==> 80km*10 ==> 10 is scaling factor
        True,  # start cumul to zero
        'Distance')

    distance_dimension = routing.GetDimensionOrDie('Distance')
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH) #to escape local minima 
    search_parameters.time_limit.FromSeconds(10)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        display = print_solution(data, manager, routing, solution)
        summary_df = save_solution(data, manager, routing, solution)
        
    return display, summary_df
        


if __name__ == '__main__':
    result_display , result_save =  main()

Route for vehicle 0:
 Depo Load(0) ->  Tharavadu old age home Load(44) ->  Alshifa Hospital Load(76) ->  CiniPlex Cinemas Cafe Load(122) ->  Gandhibhavan Snehalayam Load(160) ->  Ramees Kottiyam Load(205) ->  Dr Viswanathan Speciality Hospital Load(222) ->  YWCA Hostel for students and working women Load(255) ->  NSS Working Womens Hostel Load(268) ->  Depo Load(268)
Distance of the route: 46309.9m
Load of the route: 268

Route for vehicle 1:
 Depo Load(0) ->  Dr. Nair's Hospital Load(48) ->  SANTHAM' Avenue Load(54) ->  Children's Park Cafe Load(75) ->  Edmonds Hostel For Men Load(81) ->  Sree Anandha Bhavan Load(124) ->  HOTEL HARISREE Load(153) ->  Kilikollur Hotel Load(158) ->  MR Bakery And Juice Stall Load(205) ->  Al - Diya Bake Load(221) ->  A N Bakers & Juice Stall Load(254) ->  Sharjah Juice Corner Load(288) ->  Frankztreat Restaurant Load(323) ->  Golden Loaf Bakers Load(340) ->  Cake&bake Load(379) ->  Love Sugar Bakery Load(412) ->  Callisto cafe Load(462) ->  Taiba Hostel

In [32]:
result_save

,VehicleID,Optimized_route,Optimized_route_index,Total_load,Total_distance
0,0,"[Depo, Tharavadu old age home, Alshifa Hospita...","[0, 77, 62, 9, 81, 10, 45, 73, 64, 0]",268,463099
1,1,"[Depo, Dr. Nair's Hospital, SANTHAM' Avenue, C...","[0, 20, 70, 16, 76, 15, 39, 2, 93, 92, 97, 86,...",900,446499
2,2,"[Depo, Collectorate womens hostel, Sree Valsom...","[0, 68, 66, 17, 65, 48, 14, 13, 80, 41, 12, 71...",669,455788
3,3,"[Depo, Oottupura Restaurant, Fast Food, Kollam...","[0, 49, 44, 19, 63, 47, 57, 72, 56, 74, 40, 69...",853,453146


In [34]:
def get_coordinated_for_selected_locs(Optimized_Route_Df):

    li_coord=[]

    for route in Optimized_Route_Df['Optimized_route_index']:
        coord=[]
        for loc in route:
            if(loc)==0:
                coord.append([data_fnl.Longitude.iloc[0],data_fnl.Latitude.iloc[0]])
            else: 
                coord.append([data_fnl.Longitude.iloc[loc],data_fnl.Latitude.iloc[loc]])
        li_coord.append(coord)
    return li_coord

In [35]:
get_coordinated_for_selected_locs(result_save)

[[['8.90162924792571', '76.57825308040086'],
  ['8.921144280498778', '76.6311997012902'],
  ['8.935638304288737', '76.64914425459804'],
  ['8.929833450200626', '76.70673690417486'],
  ['8.916164685400885', '76.68831805226878'],
  ['8.86634542935278', '76.67512641268858'],
  ['8.858369061074866', '76.6527378526651'],
  ['8.884176205110224', '76.59652416252709'],
  ['8.89893127102634', '76.57832805680353'],
  ['8.90162924792571', '76.57825308040086']],
 [['8.90162924792571', '76.57825308040086'],
  ['8.899093025503628', '76.58765334842253'],
  ['8.899948840001175', '76.58227626916269'],
  ['8.896267623935751', '76.58798088961983'],
  ['8.891469013201172', '76.59394924203953'],
  ['8.888590094518493', '76.59741927095878'],
  ['8.90840125963502', '76.62369003170839'],
  ['8.908455210793479', '76.62425508719879'],
  ['8.912848149680727', '76.62871733284634'],
  ['8.914416840777937', '76.63034811586486'],
  ['8.917656063232975', '76.63250687926663'],
  ['8.917624316935022', '76.6321206420868

In [38]:
def get_route(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    
    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = requests.get(url + loc) 
    if r.status_code!= 200:
        return {}
  
    res = r.json()   
    routes = polyline.decode(res['routes'][0]['geometry'])
    start_point = [res['waypoints'][0]['location'][1], res['waypoints'][0]['location'][0]]
    end_point = [res['waypoints'][1]['location'][1], res['waypoints'][1]['location'][0]]
    distance = res['routes'][0]['distance']
    
    out = {'route':routes,
           'start_point':start_point,
           'end_point':end_point,
           'distance':distance
          }

    return out

In [39]:
def routing_mapping(x):
    routing_list = []
    individual_routes = []
    for z in range(len(x)):
        w = x.iloc[z]
        test = get_route(w.pickup_lon,w.pickup_lat,w.dropoff_lon,w.dropoff_lat)
        y = test.get('route')
        individual_routes.append(y)
        if z != (len(x)-1):
            for n in range(len(y)-1):
                routing_list.append(y[n])
        else:
            for n in range(len(y)):
                routing_list.append(y[n])
    return routing_list, individual_routes

In [49]:
#Define a folium Map
m = folium.Map(location=[data_fnl.Longitude.iloc[0],data_fnl.Latitude.iloc[0]], zoom_start=14)

#plot Depo
folium.Marker(location=[data_fnl.Longitude.iloc[0],data_fnl.Latitude.iloc[0]],
              icon=folium.Icon(icon='home', color='red')).add_to(m)

#Colors for folium route + Marker
color_route = ['blue','red','black','purple','green','orange','pink','lightblue', 'lightgreen', 'gray']

for x in range(len(result_save)):
    if len(result_save.Optimized_route_index.iloc[x]) > 1:
        routeX = pd.DataFrame(columns=['pickup','dropoff','pickup_lon','pickup_lat','dropoff_lon','dropoff_lat'])
        for i in range(len(result_save.Optimized_route_index.iloc[x])-1):
               routeX.loc[i] = [result_save.Optimized_route.iloc[x][i]] + [result_save.Optimized_route.iloc[x][i+1]] + [get_coordinated_for_selected_locs(result_save)[x][i][1]] + [get_coordinated_for_selected_locs(result_save)[x][i][0]] +  [get_coordinated_for_selected_locs(result_save)[x][i+1][1]] + [get_coordinated_for_selected_locs(result_save)[x][i+1][0]]
#routeX
        routing_list, individual_routes = routing_mapping(routeX)
        folium.PolyLine(routing_list, weight=8, color=color_route[x], opacity=0.9).add_to(m)
        for z in range(1, len(routeX)-1):
            folium.Marker(location=[routeX.pickup_lat.iloc[z], routeX.pickup_lon.iloc[z]],
                  popup=f'{z}. {routeX.pickup.iloc[z]}', icon=folium.Icon(icon='play', color=color_route[x])).add_to(m)
m
               
